In [1]:
#Importing libraries
import pandas as pd
import json
from password import password
from sqlalchemy import create_engine, MetaData ,text,inspect,table
from sqlalchemy_utils import create_database, database_exists




In [2]:
#opening the file and loading it into a variable
with open("Base.json","r") as file:
    data = json.load(file)


In [3]:
Stats= data["Statistics"]
Statsdf = pd.DataFrame.from_dict(Stats,orient="index").transpose()
Statsdf

,Total Trades,Average Win,Average Loss,Compounding Annual Return,Drawdown,Expectancy,Net Profit,Sharpe Ratio,Probabilistic Sharpe Ratio,Loss Rate,...,Beta,Annual Standard Deviation,Annual Variance,Information Ratio,Tracking Error,Treynor Ratio,Total Fees,Estimated Strategy Capacity,Lowest Capacity Asset,Portfolio Turnover
0,9655,0.14%,-0.07%,18.416%,30.100%,0.648,552.172%,1.097,54.837%,42%,...,-0.104,0.12,0.014,0.489,0.229,-1.267,$312.88,$3000.00,BRIOF R735QTJ8XC9X,0.48%


In [4]:
RunTimeStats = data["RuntimeStatistics"]
RunTimeStatsdf = pd.DataFrame.from_dict(RunTimeStats,orient="index").transpose()
RunTimeStatsdf

,Capacity,Equity,Fees,Holdings,Net Profit,Probabilistic Sharpe Ratio,Return,Unrealized,Volume
0,$3K,"$652,172.06",-$312.88,"$1,709,332.33","$598,631.21",54.837%,552.17 %,"$-46,544.61","$6,230,008.67"


In [5]:
#aquiring sub-dictionaries and loading them into a database
Rolw = data['RollingWindow'].items()
df = []
for key1 , value1 in Rolw:
    for key2, value2 in value1.items(): 
        sub_df = pd.DataFrame.from_dict(value2,orient="index").transpose()
        sub_df["date"] = key1
        df.append(sub_df.set_index(["date"]))
finaldf = pd.concat(df,axis = 0)

In [6]:
#Now we have redundant rows, better to separate them

finaldf

,StartDateTime,EndDateTime,TotalNumberOfTrades,NumberOfWinningTrades,NumberOfLosingTrades,TotalProfitLoss,TotalProfit,TotalLoss,LargestProfit,LargestLoss,...,TotalNetProfit,ProbabilisticSharpeRatio,Alpha,Beta,AnnualStandardDeviation,AnnualVariance,InformationRatio,TrackingError,TreynorRatio,PortfolioTurnover
date,,,,,,,,,,,,,,,,,,,,,
M1_20000131,None,None,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
M1_20000131,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,2.3468,0.1969,0,0
M1_20000229,None,None,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
M1_20000229,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.1947,0.0000,-0.9857,-0.4653,0.1534,0.0235,-2.1472,0.2642,1.8329,0.0650
M1_20000331,None,None,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M12_20101231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0056,0.1382,-0.0054,0.1001,0.0648,0.0042,-0.7292,0.1487,0.0601,0.0055
M12_20110131,2000-02-02T05:00:00Z,2010-07-07T04:00:00Z,745,367,378,214573.34,333096.18,-118522.84,12151.8,-9051.51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
M12_20110131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0322,0.2386,0.0147,0.0527,0.0539,0.0029,-0.8156,0.1481,0.4230,0.0055


In [7]:
# Copying every other row into a new df, and deleting the row for a second df.
portfoliodf = finaldf.iloc[1::2]
tradingdf = finaldf.iloc[::2]


In [8]:
#removing unused columns
portfoliodf = portfoliodf.iloc[:, 41:]

portfoliodf

,AverageWinRate,AverageLossRate,Expectancy,CompoundingAnnualReturn,Drawdown,TotalNetProfit,ProbabilisticSharpeRatio,Alpha,Beta,AnnualStandardDeviation,AnnualVariance,InformationRatio,TrackingError,TreynorRatio,PortfolioTurnover
date,,,,,,,,,,,,,,,
M1_20000131,0,0,0,0,0,0,0,0,0,0,0,2.3468,0.1969,0,0
M1_20000229,0,0,0,-0.9405,0.195,-0.1947,0.0000,-0.9857,-0.4653,0.1534,0.0235,-2.1472,0.2642,1.8329,0.0650
M1_20000331,0,0,0,4.1940,0.091,0.1485,0.8277,4.5611,-0.7377,0.4261,0.1816,5.0070,0.5669,-5.1915,0
M1_20000430,0,0,0,3.4600,0.073,0.1259,0.7275,1.8780,-1.1055,0.4151,0.1723,3.7974,0.6342,-1.9507,0
M1_20000531,0,0,0,0.9403,0.042,0.0560,0.7783,0.7178,-0.8007,0.1893,0.0358,2.8417,0.3935,-1.1188,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M12_20101031,0.0016,-0.0006,1.3136,0.0942,0.151,0.0939,0.4245,0.0467,0.1651,0.0775,0.0060,-0.4016,0.1456,0.4086,0.0055
M12_20101130,0.0016,-0.0006,1.3111,0.0908,0.151,0.0905,0.3977,0.0465,0.1098,0.0679,0.0046,-0.1370,0.1486,0.4987,0.0055
M12_20101231,0.0015,-0.0006,1.3012,0.0057,0.151,0.0056,0.1382,-0.0054,0.1001,0.0648,0.0042,-0.7292,0.1487,0.0601,0.0055


In [9]:
#removing unused columns
tradingdf = tradingdf.iloc[:,:41]

tradingdf

,StartDateTime,EndDateTime,TotalNumberOfTrades,NumberOfWinningTrades,NumberOfLosingTrades,TotalProfitLoss,TotalProfit,TotalLoss,LargestProfit,LargestLoss,...,ProfitLossStandardDeviation,ProfitLossDownsideDeviation,ProfitFactor,SharpeRatio,SortinoRatio,ProfitToMaxDrawdownRatio,MaximumEndTradeDrawdown,AverageEndTradeDrawdown,MaximumDrawdownDuration,TotalFees
date,,,,,,,,,,,,,,,,,,,,,
M1_20000131,None,None,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,00:00:00,0
M1_20000229,None,None,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,00:00:00,0
M1_20000331,None,None,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,00:00:00,0
M1_20000430,None,None,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,00:00:00,0
M1_20000531,None,None,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,00:00:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M12_20101031,2000-02-02T05:00:00Z,2010-07-07T04:00:00Z,747,368,379,215122.95,334158.70,-119035.75,12151.8,-9051.51,...,1323.5836,639.0040,2.8072,0.2176,0.4507,9.3816,-68422.85,-930.9591,1427.00:00:00,54.1201
M12_20101130,2000-02-02T05:00:00Z,2010-07-07T04:00:00Z,747,368,379,215122.95,334158.70,-119035.75,12151.8,-9051.51,...,1323.5836,639.0040,2.8072,0.2176,0.4507,9.3816,-68422.85,-930.9591,1427.00:00:00,54.1201
M12_20101231,2000-02-02T05:00:00Z,2010-07-07T04:00:00Z,745,367,378,214573.34,333096.18,-118522.84,12151.8,-9051.51,...,1324.7318,639.7688,2.8104,0.2174,0.4502,9.3576,-68422.85,-927.3598,00:00:00,53.9610


In [10]:
#separating Portfolio df based on months
m1portfoliodfmask = portfoliodf.index.get_level_values('date').str.contains('M1_')
m3portfoliodfmask = portfoliodf.index.get_level_values('date').str.contains('M3_')
m12portfoliodfmask = portfoliodf.index.get_level_values('date').str.contains('M12_')

m1portfoliodf = portfoliodf.loc[m1portfoliodfmask]
m3portfoliodf = portfoliodf.loc[m3portfoliodfmask]
m12portfoliodf = portfoliodf.loc[m12portfoliodfmask]

In [11]:
m1portfoliodf
m1portfoliodf = m1portfoliodf.reset_index(drop=False)
m1portfoliodf

,date,AverageWinRate,AverageLossRate,Expectancy,CompoundingAnnualReturn,Drawdown,TotalNetProfit,ProbabilisticSharpeRatio,Alpha,Beta,AnnualStandardDeviation,AnnualVariance,InformationRatio,TrackingError,TreynorRatio,PortfolioTurnover
0,M1_20000131,0,0,0,0,0,0,0,0,0,0,0,2.3468,0.1969,0,0
1,M1_20000229,0,0,0,-0.9405,0.195,-0.1947,0.0000,-0.9857,-0.4653,0.1534,0.0235,-2.1472,0.2642,1.8329,0.0650
2,M1_20000331,0,0,0,4.1940,0.091,0.1485,0.8277,4.5611,-0.7377,0.4261,0.1816,5.0070,0.5669,-5.1915,0
3,M1_20000430,0,0,0,3.4600,0.073,0.1259,0.7275,1.8780,-1.1055,0.4151,0.1723,3.7974,0.6342,-1.9507,0
4,M1_20000531,0,0,0,0.9403,0.042,0.0560,0.7783,0.7178,-0.8007,0.1893,0.0358,2.8417,0.3935,-1.1188,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,M1_20101031,0,0,0,-0.1279,0.018,-0.0112,0.2436,-0.0203,-0.1386,0.0632,0.0040,-3.3783,0.1223,0.4912,0
130,M1_20101130,0,0,0,0.2783,0.018,0.0197,0.7251,0.1793,-0.0231,0.0586,0.0034,0.8595,0.1410,-7.7156,0
131,M1_20101231,0,0,0,0.1854,0.013,0.0143,0.7724,-0.0671,0.5201,0.0506,0.0026,-5.8162,0.0496,0.3324,0
132,M1_20110131,0,0,0,0.6516,0.005,0.0427,0.9923,0.4251,0.1010,0.0510,0.0026,3.2833,0.0895,4.3545,0


In [12]:
m3portfoliodf
m3portfoliodf = m3portfoliodf.reset_index(drop=False)
m3portfoliodf

,date,AverageWinRate,AverageLossRate,Expectancy,CompoundingAnnualReturn,Drawdown,TotalNetProfit,ProbabilisticSharpeRatio,Alpha,Beta,AnnualStandardDeviation,AnnualVariance,InformationRatio,TrackingError,TreynorRatio,PortfolioTurnover
0,M3_20000131,0,0,0,0,0,0,0,0,0,0,0,2.3468,0.1969,0,0
1,M3_20000229,0,0,0,-0.7380,0.195,-0.1947,0.0000,-0.6556,-0.1660,0.1205,0.0145,-1.2978,0.2409,3.6556,0.0319
2,M3_20000331,0,0,0,-0.3948,0.301,-0.1173,0.1147,-0.2406,-0.4193,0.2851,0.0813,-0.8517,0.3955,0.6416,0.0209
3,M3_20000430,0,0,0,-0.1551,0.301,-0.0402,0.2682,0.1044,-0.8750,0.3754,0.1409,-0.3073,0.5265,0.0227,0.0209
4,M3_20000531,0,0,0,2.0340,0.091,0.3186,0.8408,1.6460,-0.9019,0.3560,0.1268,2.6386,0.5314,-1.6968,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,M3_20101031,0,0,0,-0.2715,0.081,-0.0759,0.0016,-0.2020,0.0187,0.0521,0.0027,-3.3178,0.1356,-10.5615,0
130,M3_20101130,0,0,0,-0.1293,0.058,-0.0336,0.0690,-0.0756,-0.0293,0.0572,0.0033,-3.3805,0.1239,2.9152,0
131,M3_20101231,0,0,0,0.0770,0.023,0.0188,0.5201,0.0718,-0.0250,0.0571,0.0033,-2.2238,0.1157,-2.5545,0
132,M3_20110131,0,0,0,0.3438,0.023,0.0768,0.9609,0.2177,0.0357,0.0533,0.0028,-0.2782,0.1063,6.3584,0


In [13]:
m12portfoliodf
m12portfoliodf = m12portfoliodf.reset_index(drop=False)
m12portfoliodf

,date,AverageWinRate,AverageLossRate,Expectancy,CompoundingAnnualReturn,Drawdown,TotalNetProfit,ProbabilisticSharpeRatio,Alpha,Beta,AnnualStandardDeviation,AnnualVariance,InformationRatio,TrackingError,TreynorRatio,PortfolioTurnover
0,M12_20000131,0,0,0,0,0,0,0,0,0,0,0,2.3468,0.1969,0,0
1,M12_20000229,0,0,0,-0.7380,0.195,-0.1947,0.0000,-0.6556,-0.1660,0.1205,0.0145,-1.2978,0.2409,3.6556,0.0319
2,M12_20000331,0,0,0,-0.3948,0.301,-0.1173,0.1147,-0.2406,-0.4193,0.2851,0.0813,-0.8517,0.3955,0.6416,0.0209
3,M12_20000430,0,0,0,-0.1175,0.301,-0.0402,0.2241,-0.0320,-0.6795,0.3226,0.1041,-0.0862,0.4643,0.0518,0.0157
4,M12_20000531,0,0,0,0.0329,0.301,0.0135,0.2741,0.0537,-0.7077,0.2999,0.0900,0.2022,0.4494,-0.0976,0.0125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,M12_20101031,0.0016,-0.0006,1.3136,0.0942,0.151,0.0939,0.4245,0.0467,0.1651,0.0775,0.0060,-0.4016,0.1456,0.4086,0.0055
130,M12_20101130,0.0016,-0.0006,1.3111,0.0908,0.151,0.0905,0.3977,0.0465,0.1098,0.0679,0.0046,-0.1370,0.1486,0.4987,0.0055
131,M12_20101231,0.0015,-0.0006,1.3012,0.0057,0.151,0.0056,0.1382,-0.0054,0.1001,0.0648,0.0042,-0.7292,0.1487,0.0601,0.0055
132,M12_20110131,0.0014,-0.0006,1.2994,0.0323,0.151,0.0322,0.2386,0.0147,0.0527,0.0539,0.0029,-0.8156,0.1481,0.4230,0.0055


In [14]:
#separating Trading df based on months
m1tradingdfmask = tradingdf.index.get_level_values('date').str.contains('M1_')
m3tradingdfmask = tradingdf.index.get_level_values('date').str.contains('M3_')
m12tradingdfmask = tradingdf.index.get_level_values('date').str.contains('M12_')

m1tradingdf = tradingdf.loc[m1tradingdfmask]
m3tradingdf = tradingdf.loc[m3tradingdfmask]
m12tradingdf = tradingdf.loc[m12tradingdfmask]

In [15]:
m1tradingdf
m1tradingdf = m1tradingdf.reset_index(drop=False)
m1tradingdf

,date,StartDateTime,EndDateTime,TotalNumberOfTrades,NumberOfWinningTrades,NumberOfLosingTrades,TotalProfitLoss,TotalProfit,TotalLoss,LargestProfit,...,ProfitLossStandardDeviation,ProfitLossDownsideDeviation,ProfitFactor,SharpeRatio,SortinoRatio,ProfitToMaxDrawdownRatio,MaximumEndTradeDrawdown,AverageEndTradeDrawdown,MaximumDrawdownDuration,TotalFees
0,M1_20000131,None,None,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,00:00:00,0
1,M1_20000229,None,None,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,00:00:00,0
2,M1_20000331,None,None,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,00:00:00,0
3,M1_20000430,None,None,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,00:00:00,0
4,M1_20000531,None,None,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,00:00:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,M1_20101031,None,None,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,00:00:00,0
130,M1_20101130,None,None,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,00:00:00,0
131,M1_20101231,None,None,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,00:00:00,0
132,M1_20110131,None,None,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,00:00:00,0


In [16]:
m3tradingdf
m3tradingdf = m3tradingdf.reset_index(drop=False)
m3tradingdf

,date,StartDateTime,EndDateTime,TotalNumberOfTrades,NumberOfWinningTrades,NumberOfLosingTrades,TotalProfitLoss,TotalProfit,TotalLoss,LargestProfit,...,ProfitLossStandardDeviation,ProfitLossDownsideDeviation,ProfitFactor,SharpeRatio,SortinoRatio,ProfitToMaxDrawdownRatio,MaximumEndTradeDrawdown,AverageEndTradeDrawdown,MaximumDrawdownDuration,TotalFees
0,M3_20000131,None,None,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,00:00:00,0
1,M3_20000229,None,None,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,00:00:00,0
2,M3_20000331,None,None,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,00:00:00,0
3,M3_20000430,None,None,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,00:00:00,0
4,M3_20000531,None,None,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,00:00:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,M3_20101031,None,None,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,00:00:00,0
130,M3_20101130,None,None,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,00:00:00,0
131,M3_20101231,None,None,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,00:00:00,0
132,M3_20110131,None,None,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,00:00:00,0


In [17]:
m12tradingdf
m12tradingdf = m12tradingdf.reset_index(drop=False)
m12tradingdf

,date,StartDateTime,EndDateTime,TotalNumberOfTrades,NumberOfWinningTrades,NumberOfLosingTrades,TotalProfitLoss,TotalProfit,TotalLoss,LargestProfit,...,ProfitLossStandardDeviation,ProfitLossDownsideDeviation,ProfitFactor,SharpeRatio,SortinoRatio,ProfitToMaxDrawdownRatio,MaximumEndTradeDrawdown,AverageEndTradeDrawdown,MaximumDrawdownDuration,TotalFees
0,M12_20000131,None,None,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,00:00:00,0
1,M12_20000229,None,None,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,00:00:00,0
2,M12_20000331,None,None,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,00:00:00,0
3,M12_20000430,None,None,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,00:00:00,0
4,M12_20000531,None,None,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,00:00:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,M12_20101031,2000-02-02T05:00:00Z,2010-07-07T04:00:00Z,747,368,379,215122.95,334158.70,-119035.75,12151.8,...,1323.5836,639.0040,2.8072,0.2176,0.4507,9.3816,-68422.85,-930.9591,1427.00:00:00,54.1201
130,M12_20101130,2000-02-02T05:00:00Z,2010-07-07T04:00:00Z,747,368,379,215122.95,334158.70,-119035.75,12151.8,...,1323.5836,639.0040,2.8072,0.2176,0.4507,9.3816,-68422.85,-930.9591,1427.00:00:00,54.1201
131,M12_20101231,2000-02-02T05:00:00Z,2010-07-07T04:00:00Z,745,367,378,214573.34,333096.18,-118522.84,12151.8,...,1324.7318,639.7688,2.8104,0.2174,0.4502,9.3576,-68422.85,-927.3598,00:00:00,53.9610
132,M12_20110131,2000-02-02T05:00:00Z,2010-07-07T04:00:00Z,745,367,378,214573.34,333096.18,-118522.84,12151.8,...,1324.7318,639.7688,2.8104,0.2174,0.4502,9.3576,-68422.85,-927.3598,00:00:00,53.9610


In [18]:
#Initialise connection
protocol = 'postgresql'
username = 'postgres'
password = password
host = 'localhost'
port = 5432
database_name = 'Task2'
rds_connection_string = f'{protocol}+psycopg2://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [19]:
# Check if the database exists and then create it if there are none
if not database_exists(engine.url):
    create_database(engine.url)

In [20]:
#Checking connection
with engine.connect() as conn:
    sql = text('SELECT current_database()')
    result = conn.execute(sql)
    print(result.fetchone()[0])  # Should print "Task2"

Task2


In [21]:
#Loading dfs to SQL
m1portfoliodf.to_sql(name='PortfolioStatsM1', con=engine, if_exists='replace', index=False)
m3portfoliodf.to_sql(name='PortfolioStatsM3', con=engine, if_exists='replace', index=False)
m12portfoliodf.to_sql(name='PortfolioStatsM12', con=engine, if_exists='replace', index=False)
m1tradingdf.to_sql(name='TradingStatsM1', con=engine, if_exists='replace', index=False)
m3tradingdf.to_sql(name='TradingStatsM3', con=engine, if_exists='replace', index=False)
m12tradingdf.to_sql(name='TradingStatsM12', con=engine, if_exists='replace', index=False)

Statsdf.to_sql(name='Stats', con=engine, if_exists='replace', index=False)
RunTimeStatsdf.to_sql(name='RunTimeStats', con=engine, if_exists='replace', index=False)

1

In [22]:
#Checking tables names
inspector = inspect(engine)
table_names = inspector.get_table_names()
print(table_names)

['PortfolioStatsM3', 'PortfolioStatsM12', 'PortfolioStatsM1', 'TradingStatsM1', 'TradingStatsM3', 'TradingStatsM12', 'Stats', 'RunTimeStats']
